# Training classifiers on HOG Data
#### For extraction of features, see extractFeatures.ipynb

conda env- make sure to switch to python3 kernel, faus_dl

## Importing Folders

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sklearn
from sklearn import datasets, svm, metrics
import bcolz
import os, sys
# import datetime as dt
# import random

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
import pickle

In [ ]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

In [19]:
model1=pickle.load(open("/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hog_FAU4_1_o_6_ppc_8_cpb_4_thresh_3",'rb'))

In [20]:
test_data=np.load("array2.npy")
test_data.shape

(91, 1056)

In [22]:
model1.predict(test_data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [26]:
model1=pickle.load(open("/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hog_FAU2_1_o_6_ppc_8_cpb_4_thresh_3",'rb'))

In [27]:
test_data=np.load("array3.npy")
test_data.shape

(43, 1056)

In [28]:
model1.predict(test_data).sum()

0

## Training Details explained

Loading the saved features:

In [5]:
dir_features_hog_1_fau4_1=DISFA_path+"/features/hog_1/fau4_1/"
dir_features_hog_1_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_1/fau4_1/'

In [6]:
features_fau4_1_hog_1_positives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives/',mode='r')
print("features_fau4_1_hog_1_positives loaded, shape is: ",features_fau4_1_hog_1_positives.shape)

features_fau4_1_hog_1_positives loaded, shape is:  (2000, 448)


In [7]:
features_fau4_1_hog_1_negatives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives/',mode='r')
print("features_fau4_1_hog_1_negatives loaded, shape is : ",features_fau4_1_hog_1_negatives.shape)

features_fau4_1_hog_1_negatives loaded, shape is :  (1898, 448)


Loading test_features:

In [8]:
features_fau4_1_hog_1_positives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives_test/',mode='r')
print("features_fau4_1_hog_1_positives_test loaded, shape is: ",features_fau4_1_hog_1_positives_test.shape)

features_fau4_1_hog_1_positives_test loaded, shape is:  (464, 448)


In [9]:
features_fau4_1_hog_1_negatives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives_test/',mode='r')
print("features_fau4_1_hog_1_negatives_test loaded, shape is : ",features_fau4_1_hog_1_negatives_test.shape)

features_fau4_1_hog_1_negatives_test loaded, shape is :  (499, 448)


Defining the training and testing data

In [10]:
train_no=1800
X=np.concatenate([features_fau4_1_hog_1_positives[:train_no],features_fau4_1_hog_1_negatives[:train_no]])
X.shape

(3600, 448)

In [11]:
Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
Y.shape

(3600,)

In [12]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size=0.15,random_state=42)

Defining the parameter range for SVM:

In [13]:
gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
gamma_range=gamma_range.flatten()
C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
C_range=C_range.flatten()
parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}

In [14]:
svm_clsf=svm.SVC()
grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=8,verbose=2)

Start training and parameter search 

In [15]:
start_time=dt.datetime.now()
print('Start param searching at {}'.format(str(start_time)))
grid_clsf.fit(X_train,y_train)
elapsed_time=dt.datetime.now()-start_time
print('Elapsed time, param searching {}'.format(str(elapsed_time)))
sorted(grid_clsf.cv_results_.keys())

Start param searching at 2018-06-07 10:23:59.808611
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=  11.2s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=  12.3s
[CV] C=0.1, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.1, ga

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.2min


[CV] ................... C=0.5, gamma=0.001, kernel=rbf, total=  13.0s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  11.1s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  12.5s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] .................... C=0.5, gamma=0.01, kernel=rbf, total=  10.3s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  11.7s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] .................... C=0.5, gamma=0.01, kernel=rbf, total=  10.7s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] ................... C=0.5, gamma=0.001, kernel=rbf, total=  14.4s
[CV] C=0.5, gamma=0.5, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=8)]: Done 144 out of 144 | elapsed:  4.5min finished


Elapsed time, param searching 0:04:36.259007


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [16]:
classifier=grid_clsf.best_estimator_
params=grid_clsf.best_params_

scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
# plot_param_space_scores(scores,C_range, gamma_range)

In [17]:
scores

array([[0.50915033, 0.60130719, 0.78398693, 0.81143791, 0.83529412,
        0.83267974, 0.8003268 , 0.50915033],
       [0.60163399, 0.75980392, 0.81699346, 0.85784314, 0.88660131,
        0.92614379, 0.91862745, 0.50915033],
       [0.78039216, 0.81601307, 0.84019608, 0.89509804, 0.92287582,
        0.94411765, 0.93888889, 0.61437908],
       [0.81699346, 0.86699346, 0.89509804, 0.94248366, 0.95326797,
        0.95490196, 0.95163399, 0.64379085],
       [0.84215686, 0.89542484, 0.91699346, 0.9503268 , 0.95196078,
        0.95490196, 0.95163399, 0.64379085],
       [0.89705882, 0.92745098, 0.93529412, 0.94542484, 0.94803922,
        0.95490196, 0.95163399, 0.64379085]])

In [18]:
expected = y_test
predicted = classifier.predict(X_test)
print(predicted)
print("predicted and expected are same in: ", np.sum(expected==predicted)," out of ", expected.shape[0])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
      
cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

# plot_confusion_matrix(cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


[1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1
 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0
 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1
 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1
 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1
 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0
 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0
 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 1 0 0 0 0 1 0 

In [19]:
print(classifier.predict(features_fau4_1_hog_1_positives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

14  out of  464


In [20]:
print(classifier.predict(features_fau4_1_hog_1_negatives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

0  out of  464


**Saving the model to disk**

In [32]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [33]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [34]:
with open(models_path+'fau4_1_hog_1.pickle','wb') as handle:
    pickle.dump(classifier,handle)

**Loading a model from disk**

In [35]:
with open(models_path+'fau4_1_hog_1.pickle','rb') as handle:
    clf1=pickle.load(handle)

**Calculating features for a cropped image from disk**

In [23]:
dir_cropped_FAU4_1=DISFA_path+"/features/cropped_images/FAU4_1/"
print(dir_cropped_FAU4_1)
dir_cropped_FAU4_1_positives=dir_cropped_FAU4_1+"positives/"
print(dir_cropped_FAU4_1_positives)
dir_cropped_FAU4_1_negatives=dir_cropped_FAU4_1+"negatives/"
print(dir_cropped_FAU4_1_negatives)

/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/positives/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/negatives/


In [24]:
list_cropped_fau4_1_positive_files=os.listdir(dir_cropped_FAU4_1_positives)
list_cropped_fau4_1_positive_files[0]

'RightVideoSN001_1340.png'

In [44]:
classifier.predict(features_fau4_1_hog_1_positives)

array([1, 1, 1, ..., 1, 1, 1])

## Old Trained Model (before June 8)

In [5]:
dir_features_hog_2_fau4_1=DISFA_path+"/features/hog_2/fau4_1/"
dir_features_hog_2_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_2/fau4_1/'

Training or FAU 4 HOG2(Affectiva configuration)

**RBF SVM training**

In [ ]:
X,Y=loadTrainData(dir_features_hog_2_fau4_1)
best_classifier,param,scores=trainModel(X,Y)

In [21]:
features_folder=dir_features_hog_2_fau4_1
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (469, 1056)
features_negatives_test loaded, shape is:  (498, 1056)


In [26]:
print (best_classifier.predict(features_positives_test).sum(), " out of ", len(features_positives_test),best_classifier.predict(features_positives_test).sum()/len(features_positives_test) )

294  out of  469 0.6268656716417911


In [29]:
print (best_classifier.predict(features_negatives_test).sum(), " out of ", len(features_negatives_test),(len(features_negatives_test)-best_classifier.predict(features_negatives_test).sum())/len(features_negatives_test))

108  out of  498 0.7831325301204819


In [35]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [36]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [38]:
with open(models_path+'fau4_1_hog_2_rbf.pickle','wb') as handle:
    pickle.dump(best_classifier,handle)

**Linear SVM training**

In [45]:
def trainModel(X_train,y_train):
    #setup
    gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
    gamma_range=gamma_range.flatten()
    C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
    C_range=C_range.flatten()
    parameters={'kernel':['linear'],'C':C_range,'gamma':gamma_range}
    svm_clsf=svm.SVC()
    grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)
    #train
    start_time=dt.datetime.now()
    print('Start param searching at {}'.format(str(start_time)))
    grid_clsf.fit(X_train,y_train)
    elapsed_time=dt.datetime.now()-start_time
    print('Elapsed time, param searching {}'.format(str(elapsed_time)))
    sorted(grid_clsf.cv_results_.keys())
    #choose classifier
    classifier=grid_clsf.best_estimator_
    params=grid_clsf.best_params_
    scores=grid_clsf.cv_results_['mean_test_score'].reshape(2,len(C_range),len(gamma_range))
    print("scores are: ", scores)
    return classifier, params,scores

In [31]:
X,Y=loadTrainData(dir_features_hog_2_fau4_1)
best_classifier,param,scores=trainModel(X,Y)

features_positives loaded, shape is:  (1999, 1056)
features_negatives loaded, shape is :  (2000, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-06 16:47:41.880015
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.4s remaining:    0.0s


[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=   9.5s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=   9.8s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=  10.2s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=  10.2s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=   9.9s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 24.1min finished


Elapsed time, param searching 0:24:08.732567
scores are:  [[0.74462231 0.74462231 0.74462231 0.74462231 0.74462231 0.74462231
  0.74462231 0.74462231]
 [0.80490245 0.80490245 0.80490245 0.80490245 0.80490245 0.80490245
  0.80490245 0.80490245]
 [0.84242121 0.84242121 0.84242121 0.84242121 0.84242121 0.84242121
  0.84242121 0.84242121]
 [0.87818909 0.87818909 0.87818909 0.87818909 0.87818909 0.87818909
  0.87818909 0.87818909]
 [0.89294647 0.89294647 0.89294647 0.89294647 0.89294647 0.89294647
  0.89294647 0.89294647]
 [0.8991996  0.8991996  0.8991996  0.8991996  0.8991996  0.8991996
  0.8991996  0.8991996 ]]


In [32]:
features_folder=dir_features_hog_2_fau4_1
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (469, 1056)
features_negatives_test loaded, shape is:  (498, 1056)


In [44]:
print (best_classifier.predict(features_positives_test).sum(), " out of ", len(features_positives_test),best_classifier.predict(features_positives_test).sum()/len(features_positives_test) )

407  out of  469 0.8678038379530917


In [43]:
print (best_classifier.predict(features_negatives_test).sum(), " out of ", len(features_negatives_test),(len(features_negatives_test)-best_classifier.predict(features_negatives_test).sum())/len(features_negatives_test))

154  out of  498 0.6907630522088354


In [39]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [40]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [42]:
with open(models_path+'fau4_1_hog_2_linear.pickle','wb') as handle:
    pickle.dump(best_classifier,handle)

## Automating loading features, training process 

In [4]:
def loadTrainData(features_folder,train_no,boolShuffle=True):
    #loading features
    features_positives=bcolz.carray(rootdir=features_folder+'positives/',mode='r')
    print("features_positives loaded, shape is: ",features_positives.shape)
    features_negatives=bcolz.carray(rootdir=features_folder+'negatives/',mode='r')
    print("features_negatives loaded, shape is : ",features_negatives.shape)
    
#     features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
#     print("features_positives_test loaded, shape is: ",features_positives_test.shape)
#     features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
#     print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)
    
    #setting up the training data
    train_no=np.min([features_positives.shape[0],features_negatives.shape[0],train_no])
#     if boolShuffle:
#         random.shuffle(features_positives)
#         random.shuffle(features_negatives)
    X=np.concatenate([features_positives[:train_no],features_negatives[:train_no]])
    print("training data shape is: ",X.shape)
    #setting labels/targets
    Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
    print("labels shape is: ",Y.shape)
    return X, Y

In [5]:
def trainModel(X_train,y_train,no_jobs):
    #setup
    gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
    gamma_range=gamma_range.flatten()
    C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
    C_range=C_range.flatten()
    parameters={'kernel':['rbf','linear'],'C':C_range,'gamma':gamma_range}
    svm_clsf=svm.SVC()
    grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=no_jobs,verbose=2)
    #train
    start_time=dt.datetime.now()
    print('Start param searching at {}'.format(str(start_time)))
    grid_clsf.fit(X_train,y_train)
    elapsed_time=dt.datetime.now()-start_time
    print('Elapsed time, param searching {}'.format(str(elapsed_time)))
    sorted(grid_clsf.cv_results_.keys())
    #choose classifier
    classifier=grid_clsf.best_estimator_
    params=grid_clsf.best_params_
    scores=grid_clsf.cv_results_['mean_test_score'].reshape(2,len(C_range),len(gamma_range))
    print("scores are: ", scores)
    return classifier, params,scores

In [6]:
def trainFinal(features_folder,no_jobs,train_no=2000):
    X,Y=loadTrainData(features_folder,train_no,True)
    classifier,params,scores=trainModel(X,Y,no_jobs)
    print(scores)
    return classifier, params,scores

In [7]:
def test(features_folder,classifier):
    features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
    print("features_positives_test loaded, shape is: ",features_positives_test.shape)
    features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
    print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)
    print (classifier.predict(features_positives_test).sum(), " positives out of ", len(features_positives_test),classifier.predict(features_positives_test).sum()/len(features_positives_test) )
    print (classifier.predict(features_negatives_test).sum(), " positives out of ", len(features_negatives_test),(len(features_negatives_test)-classifier.predict(features_negatives_test).sum())/len(features_negatives_test))

In [8]:
features_folder_path='/features/hog/3/hog_FAU2_1_o_6_ppc_8_cpb_4/'
features_folder_path.split('/')

['', 'features', 'hog', '3', 'hog_FAU2_1_o_6_ppc_8_cpb_4', '']

In [9]:
def saveModel(classifier,features_folder):
    #remember to have / at the end of features_folder name
    models_path=DISFA_path+'models/'
    if not os.path.exists(models_path):
        os.makedirs(models_path)
    filename=features_folder.split('/')[-2]
    thresh=features_folder.split('/')[-3]
    dest_file=models_path+filename+f'_thresh_{thresh}'
    with open(dest_file,'wb') as handle:
        pickle.dump(classifier,handle)
    print("model saved at: ", dest_file)

In [10]:
def trainAndSaveModel(folder_name,no_jobs=8,train_no=2000):
    clf,params,scores=trainFinal(folder_name,no_jobs,train_no)
    test(folder_name,clf)
    saveModel(clf, folder_name)

**Sample training for FAU2**

In [49]:
hog_fau2_1_folder=DISFA_path+'/features/hog_FAU2_1_o_6_ppc_8_cpb_4/'

classifier_fau2,params_fau2,scores_fau2=trainFinal(hog_fau2_1_folder,8,2000)

features_positives loaded, shape is:  (2000, 1056)
features_negatives loaded, shape is :  (1999, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-09 00:25:54.905122
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  25.1s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  3.4min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  41.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  25.6s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  28.2s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  26.8s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  27.4s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  26.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  40.4s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 13.6min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=  17.5s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  16.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  15.7s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  41.2s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  40.9s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  34.3s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  43.8s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 22.1min finished


Elapsed time, param searching 0:22:10.577138
scores are:  [[[0.75537769 0.8004002  0.75537769 0.8004002  0.75487744 0.8004002
   0.75512756 0.8004002 ]
  [0.75637819 0.8004002  0.7993997  0.8004002  0.80315158 0.8004002
   0.79989995 0.8004002 ]
  [0.75537769 0.80490245 0.75537769 0.80490245 0.75487744 0.80490245
   0.79564782 0.80490245]
  [0.8004002  0.80490245 0.80465233 0.80490245 0.80390195 0.80490245
   0.82491246 0.80490245]
  [0.75537769 0.80215108 0.75537769 0.80215108 0.75712856 0.80215108
   0.8004002  0.80215108]
  [0.80390195 0.80215108 0.8024012  0.80215108 0.80365183 0.80215108
   0.83491746 0.80215108]]

 [[0.75537769 0.81490745 0.79564782 0.81490745 0.8004002  0.81490745
   0.80515258 0.81490745]
  [0.80190095 0.81490745 0.82041021 0.81490745 0.83091546 0.81490745
   0.85642821 0.81490745]
  [0.75787894 0.82841421 0.8004002  0.82841421 0.80415208 0.82841421
   0.80165083 0.82841421]
  [0.80015008 0.82841421 0.83291646 0.82841421 0.84017009 0.82841421
   0.86543272 0.82

In [90]:
features_folder=hog_fau1_1_folder
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (504, 1056)
features_negatives_test loaded, shape is:  (510, 1056)


In [91]:
#testing

print (classifier_fau2.predict(features_positives_test).sum(), " out of ", len(features_positives_test),classifier_fau2.predict(features_positives_test).sum()/len(features_positives_test) )

print (classifier_fau2.predict(features_negatives_test).sum(), " positives out of ", len(features_negatives_test),(len(features_negatives_test)-classifier_fau2.predict(features_negatives_test).sum())/len(features_negatives_test))

494  out of  504 0.9801587301587301
72  positives out of  510 0.8588235294117647


In [61]:
hog_fau1_1_folder=DISFA_path+'/features/hog/3/hog_FAU2_1_o_6_ppc_8_cpb_4/'

classifier_fau1,params_fau1,scores_fau1=trainFinal(hog_fau1_1_folder,8,2000)

features_positives loaded, shape is:  (410, 1056)
features_negatives loaded, shape is :  (436, 1056)
training data shape is:  (820, 1056)
labels shape is:  (820,)
Start param searching at 2018-06-09 17:31:46.978564
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=   1.6s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ........

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   12.3s


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=   2.2s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.9s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.9s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=   2.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.9s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=   1.8s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=   2.0s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   41.2s


[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   1.7s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   2.1s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   2.2s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=   2.0s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed:   54.9s finished


In [62]:
test(hog_fau2_1_folder)

features_positives_test loaded, shape is:  (504, 1056)
features_negatives_test loaded, shape is:  (510, 1056)
494  positives out of  504 0.9801587301587301
72  positives out of  510 0.8588235294117647


In [94]:
saveModel(classifier_fau2,hog_fau2_1_folder)

## Training and testing

#### FAU1 threshold 3

In [101]:
features_folder_path=DISFA_path+'/features/hog/3/hog_FAU1_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (1361, 1056)
features_negatives loaded, shape is :  (1363, 1056)
training data shape is:  (2722, 1056)
labels shape is:  (2722,)
Start param searching at 2018-06-09 23:06:59.397456
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  11.2s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.7min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  21.0s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  22.0s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  11.8s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  12.5s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  12.0s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  21.7s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  13.4s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  5.7min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=   2.0s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   2.0s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   1.9s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   2.1s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=5.0, gamma=0.01, kernel=rbf, total=  11.5s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  16.0s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  15.6s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed:  7.8min finished


Elapsed time, param searching 0:07:49.969254
scores are:  [[[0.89346069 0.91219691 0.89346069 0.91219691 0.89346069 0.91219691
   0.89346069 0.91219691]
  [0.89639971 0.91219691 0.90999265 0.91219691 0.9331374  0.91219691
   0.98457017 0.91219691]
  [0.89346069 0.97097722 0.89346069 0.97097722 0.89346069 0.97097722
   0.89897134 0.97097722]
  [0.91219691 0.97097722 0.97097722 0.97097722 0.98346804 0.97097722
   0.99118295 0.97097722]
  [0.89346069 0.98273328 0.89346069 0.98273328 0.89603233 0.98273328
   0.91182954 0.98273328]
  [0.93607641 0.98273328 0.98310066 0.98273328 0.98603968 0.98273328
   0.99338722 0.98273328]]

 [[0.89346069 0.98971345 0.89823659 0.98971345 0.91219691 0.98971345
   0.97171198 0.98971345]
  [0.98310066 0.98971345 0.99191771 0.98971345 0.99301984 0.98971345
   0.99779574 0.98971345]
  [0.89603233 0.99081558 0.91219691 0.99081558 0.93681117 0.99081558
   0.98310066 0.99081558]
  [0.98530492 0.99081558 0.99191771 0.99081558 0.99559148 0.99081558
   0.99706098 0.

#### FAU4 threshold 3

In [103]:
features_folder_path=DISFA_path+'/features/hog/3/hog_FAU4_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (2000, 1056)
features_negatives loaded, shape is :  (1999, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-09 23:19:02.420621
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  34.0s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  4.0min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  42.7s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  29.6s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  29.8s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  30.4s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  31.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  32.1s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  41.2s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 16.2min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=  15.4s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  11.2s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  10.8s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  10.4s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  46.8s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  48.2s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  49.4s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 22.6min finished


Elapsed time, param searching 0:22:38.685086
scores are:  [[[0.69584792 0.75487744 0.69584792 0.75487744 0.69584792 0.75487744
   0.69584792 0.75487744]
  [0.70110055 0.75487744 0.75162581 0.75487744 0.81415708 0.75487744
   0.90145073 0.75487744]
  [0.69584792 0.87143572 0.69584792 0.87143572 0.69584792 0.87143572
   0.72036018 0.87143572]
  [0.75487744 0.87143572 0.87218609 0.87143572 0.91370685 0.87143572
   0.95022511 0.87143572]
  [0.69584792 0.9112056  0.69584792 0.9112056  0.70185093 0.9112056
   0.75487744 0.9112056 ]
  [0.8184092  0.9112056  0.91170585 0.9112056  0.93796898 0.9112056
   0.96398199 0.9112056 ]]

 [[0.69584792 0.94622311 0.72011006 0.94622311 0.75487744 0.94622311
   0.87068534 0.94622311]
  [0.9112056  0.94622311 0.95047524 0.94622311 0.96198099 0.94622311
   0.97473737 0.94622311]
  [0.70210105 0.95422711 0.75487744 0.95422711 0.8184092  0.95422711
   0.91045523 0.95422711]
  [0.93346673 0.95422711 0.96123062 0.95422711 0.96873437 0.95422711
   0.97623812 0.95

#### FAU2 threshold 3

In [104]:
features_folder_path=DISFA_path+'/features/hog/3/hog_FAU2_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (410, 1056)
features_negatives loaded, shape is :  (436, 1056)
training data shape is:  (820, 1056)
labels shape is:  (820,)
Start param searching at 2018-06-09 23:43:14.015128
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=   1.5s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ........

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   11.4s


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=   1.9s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=   1.6s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=   1.6s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=   1.5s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=   1.4s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   40.5s


[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   0.3s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   2.0s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   2.2s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=   2.1s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=   2.0s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed:   56.3s finished


Elapsed time, param searching 0:00:56.614779
scores are:  [[[0.91097561 0.92682927 0.91097561 0.92682927 0.91219512 0.92682927
   0.91219512 0.92682927]
  [0.91097561 0.92682927 0.92804878 0.92682927 0.91463415 0.92682927
   0.90243902 0.92682927]
  [0.91097561 0.93902439 0.91097561 0.93902439 0.91219512 0.93902439
   0.9097561  0.93902439]
  [0.92804878 0.93902439 0.93902439 0.93902439 0.93902439 0.93902439
   0.92926829 0.93902439]
  [0.91097561 0.94268293 0.91097561 0.94268293 0.91219512 0.94268293
   0.92682927 0.94268293]
  [0.91585366 0.94268293 0.94390244 0.94268293 0.94268293 0.94268293
   0.92926829 0.94268293]]

 [[0.91097561 0.94878049 0.9097561  0.94878049 0.92682927 0.94878049
   0.93902439 0.94878049]
  [0.94268293 0.94878049 0.94634146 0.94878049 0.94878049 0.94878049
   0.94268293 0.94878049]
  [0.91097561 0.95365854 0.92682927 0.95365854 0.91585366 0.95365854
   0.94268293 0.95365854]
  [0.95121951 0.95365854 0.95365854 0.95365854 0.94756098 0.95365854
   0.94512195 0.

#### FAU1 threshold 2

In [11]:
features_folder_path=DISFA_path+'/features/hog/2/hog_FAU1_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (1995, 1056)
features_negatives loaded, shape is :  (1992, 1056)
training data shape is:  (3984, 1056)
labels shape is:  (3984,)
Start param searching at 2018-06-10 09:31:56.873503
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  30.8s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  3.5min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  40.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  28.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  28.4s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  29.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  30.9s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  31.6s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  41.2s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 14.6min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=  11.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  11.7s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   9.7s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  10.6s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  47.1s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  48.7s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  38.8s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 21.6min finished


Elapsed time, param searching 0:21:40.200338
scores are:  [[[0.79091365 0.8564257  0.79091365 0.8564257  0.79091365 0.8564257
   0.79116466 0.8564257 ]
  [0.79241968 0.8564257  0.8564257  0.8564257  0.88253012 0.8564257
   0.9246988  0.8564257 ]
  [0.79091365 0.91591365 0.79091365 0.91591365 0.79091365 0.91591365
   0.83709839 0.91591365]
  [0.8564257  0.91591365 0.91541165 0.91591365 0.9312249  0.91591365
   0.96134538 0.91591365]
  [0.79091365 0.92745984 0.79091365 0.92745984 0.79241968 0.92745984
   0.85692771 0.92745984]
  [0.88604418 0.92745984 0.92971888 0.92745984 0.9437751  0.92745984
   0.96561245 0.92745984]]

 [[0.79091365 0.94653614 0.8376004  0.94653614 0.85692771 0.94653614
   0.91591365 0.94653614]
  [0.92771084 0.94653614 0.9503012  0.94653614 0.95883534 0.94653614
   0.96611446 0.94653614]
  [0.79216867 0.9497992  0.85667671 0.9497992  0.88478916 0.9497992
   0.92771084 0.9497992 ]
  [0.93950803 0.9497992  0.95481928 0.9497992  0.96134538 0.9497992
   0.96736948 0.9497

#### FAU4 threshold 2

In [12]:
features_folder_path=DISFA_path+'/features/hog/2/hog_FAU4_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (1932, 1056)
features_negatives loaded, shape is :  (2000, 1056)
training data shape is:  (3864, 1056)
labels shape is:  (3864,)
Start param searching at 2018-06-10 09:53:39.003469
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  31.4s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  3.8min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  44.0s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  30.0s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  30.7s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  31.4s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  33.7s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  34.7s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  42.6s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 15.2min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=  12.2s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  10.8s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  13.6s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  12.0s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  44.7s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  44.0s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  44.7s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 22.5min finished


Elapsed time, param searching 0:22:30.779065
scores are:  [[[0.75543478 0.8139234  0.75543478 0.8139234  0.75517598 0.8139234
   0.75569358 0.8139234 ]
  [0.75672878 0.8139234  0.81599379 0.8139234  0.82841615 0.8139234
   0.85921325 0.8139234 ]
  [0.75543478 0.86283644 0.75543478 0.86283644 0.75517598 0.86283644
   0.80797101 0.86283644]
  [0.81469979 0.86283644 0.85895445 0.86283644 0.87888199 0.86283644
   0.92132505 0.86283644]
  [0.75543478 0.88405797 0.75543478 0.88405797 0.75517598 0.88405797
   0.81444099 0.88405797]
  [0.82738095 0.88405797 0.88224638 0.88405797 0.90786749 0.88405797
   0.93452381 0.88405797]]

 [[0.75543478 0.92727743 0.80693582 0.92727743 0.81444099 0.92727743
   0.86231884 0.92727743]
  [0.88328157 0.92727743 0.93012422 0.92727743 0.94021739 0.92727743
   0.95574534 0.92727743]
  [0.75543478 0.93581781 0.81444099 0.93581781 0.82660455 0.93581781
   0.88354037 0.93581781]
  [0.9068323  0.93581781 0.93969979 0.93581781 0.94849896 0.93581781
   0.95626294 0.93

#### FAU2 threshold 2

In [13]:
features_folder_path=DISFA_path+'/features/hog/2/hog_FAU2_1_o_6_ppc_8_cpb_4/'
trainAndSaveModel(folder_name=features_folder_path,no_jobs=8,train_no=2000)

features_positives loaded, shape is:  (1766, 1056)
features_negatives loaded, shape is :  (2000, 1056)
training data shape is:  (3532, 1056)
labels shape is:  (3532,)
Start param searching at 2018-06-10 10:16:12.490414
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  19.0s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.9min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  35.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  34.7s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  20.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  21.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  36.8s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  21.4s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  23.4s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 10.2min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=   6.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   6.0s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   7.3s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=   6.2s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  27.8s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  26.7s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  36.7s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 14.5min finished


Elapsed time, param searching 0:14:32.295792
scores are:  [[[0.85843715 0.91987542 0.85843715 0.91987542 0.85843715 0.91987542
   0.85872027 0.91987542]
  [0.86211778 0.91987542 0.9184598  0.91987542 0.91449604 0.91987542
   0.92270668 0.91987542]
  [0.85843715 0.92383918 0.85843715 0.92383918 0.85843715 0.92383918
   0.90288788 0.92383918]
  [0.92044168 0.92383918 0.92440544 0.92383918 0.93261608 0.92383918
   0.96630804 0.92383918]
  [0.85843715 0.93714609 0.85843715 0.93714609 0.86240091 0.93714609
   0.9195923  0.93714609]
  [0.91591166 0.93714609 0.93573046 0.93714609 0.9515855  0.93714609
   0.9714043  0.93714609]]

 [[0.85843715 0.96574179 0.90458664 0.96574179 0.91987542 0.96574179
   0.92383918 0.96574179]
  [0.93714609 0.96574179 0.96602492 0.96574179 0.96942242 0.96574179
   0.9702718  0.96574179]
  [0.86268403 0.9680068  0.91987542 0.9680068  0.91591166 0.9680068
   0.93714609 0.9680068 ]
  [0.95328426 0.9680068  0.96828992 0.9680068  0.96885617 0.9680068
   0.97083805 0.96